## Running conv-nets
This ipython notebook walks you through the running_template.py script.

First, we need to load the necessary python modules

In [1]:
import h5py
import tifffile as tiff
#from keras.backend.common import _UID_PREFIXES

from cnn_functions import nikon_getfiles, get_image, run_models_on_directory, get_image_sizes, segment_nuclei, segment_cytoplasm, dice_jaccard_indices
from model_zoo import sparse_bn_feature_net_61x61 as cyto_fn
from model_zoo import sparse_bn_feature_net_61x61 as nuclear_fn

import os
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: GeForce GTX 1080 (CNMeM is disabled, cuDNN 5005)
Using Theano backend.


Next, we need to specify some directory locations - namely where the home data directory is (direc_name), where the raw images are located (data_location), where to store they cytoplasm prediction images (cyto_location), where to store the nuclear prediction images (nuclear_location), and where to store the final segmentation masks (mask_location)

In [2]:
direc_name = '/home/vanvalen/DeepCell/validation_data/HeLa/'
data_location = os.path.join(direc_name, 'RawImages')

cyto_location = os.path.join(direc_name, 'Cytoplasm')
nuclear_location = os.path.join(direc_name, 'Nuclear')
mask_location = os.path.join(direc_name, 'Masks')

Next, we need to define the channel names. In this case, the cytoplasm network takes in phase and nuclear marker (far red) images. The channel names have to be present in the file names. We also need to specify the directory the conv-net parameter files live in and what the file names are.

In [3]:
cyto_channel_names = ['phase', 'farred']
nuclear_channel_names = ['farred']

trained_network_cyto_directory = "/home/vanvalen/DeepCell/trained_networks/HeLa/"
trained_network_nuclear_directory = "/home/vanvalen/DeepCell/trained_networks/Nuclear"

cyto_prefix = "2017-06-21_HeLa_all_61x61_bn_feature_net_61x61_"
nuclear_prefix = "2016-07-12_nuclei_all_61x61_bn_feature_net_61x61_"

We are using a network that takes in 61x61 images in this example. We need to manually feed in the window size (the number of pixels sampled around each pixels). We also need to specify the image size.

In [4]:
win_cyto = 30
win_nuclear = 30

image_size_x, image_size_y = get_image_sizes(data_location, nuclear_channel_names)


Now we need to load the weights for the networks. Remember we make use of two networks - one for the cytoplasm and another for the nucleus. Because we use model parallelism, we have 5 networks for each segmentation task.

In [5]:
list_of_cyto_weights = []
for j in xrange(5):
	cyto_weights = os.path.join(trained_network_cyto_directory,  cyto_prefix + str(j) + ".h5")
	list_of_cyto_weights += [cyto_weights]

list_of_nuclear_weights = []
for j in xrange(5):
	nuclear_weights = os.path.join(trained_network_nuclear_directory,  nuclear_prefix + str(j) + ".h5")
	list_of_nuclear_weights += [nuclear_weights]

Next, we run all our networks on all the files in our directory.

In [6]:
cytoplasm_predictions = run_models_on_directory(data_location, cyto_channel_names, cyto_location, model_fn = cyto_fn, 
	list_of_weights = list_of_cyto_weights, image_size_x = image_size_x, image_size_y = image_size_y, 
	win_x = win_cyto, win_y = win_cyto, split = False)

nuclear_predictions = run_models_on_directory(data_location, nuclear_channel_names, nuclear_location, model_fn = nuclear_fn, 
	list_of_weights = list_of_nuclear_weights, image_size_x = image_size_x, image_size_y = image_size_y, 
	win_x = win_nuclear, win_y = win_nuclear, split = False)

cnn_functions.py:258: UserWarning: DEPRECATION: the 'st' parameter is not going to exist anymore as it is going to be replaced by the parameter 'stride'.
  pooled_array +=[pool_2d(input_image[:, :, offset_x::stride, offset_y::stride], pool_size, st = (1,1), mode = mode, padding = (0,0), ignore_border = True)]
cnn_functions.py:258: UserWarning: DEPRECATION: the 'padding' parameter is not going to exist anymore as it is going to be replaced by the parameter 'pad'.
  pooled_array +=[pool_2d(input_image[:, :, offset_x::stride, offset_y::stride], pool_size, st = (1,1), mode = mode, padding = (0,0), ignore_border = True)]


Processing image 1 of 1
Processing image 1 of 1
Processing image 1 of 1
Processing image 1 of 1
Processing image 1 of 1
Processing image 1 of 1
Processing image 1 of 1
Processing image 1 of 1
Processing image 1 of 1
Processing image 1 of 1


Next, we need to refine the neural network predictions to create segmentation masks. The smoothing and num_iters parameters control the active contour refinement process.

In [7]:
nuclear_masks = segment_nuclei(nuclear_predictions, mask_location = mask_location, threshold = 0.75, area_threshold = 100, solidity_threshold = 0.75, eccentricity_threshold = 0.95)
cytoplasm_masks = segment_cytoplasm(cytoplasm_predictions, nuclear_masks = nuclear_masks, mask_location = mask_location, smoothing = 1, num_iters = 120)

With the masks computed, if a reference segmentation is available, we can test our conv-net segmentation against them to compute the Jaccard and Dice indices.

In [8]:
direc_val = os.path.join(direc_name, 'Validation')
imglist_val = nikon_getfiles(direc_val, 'validation_interior')

val_name = os.path.join(direc_val, imglist_val[0]) 
val = get_image(val_name)
val = val[win_cyto:-win_cyto,win_cyto:-win_cyto]
cyto = cytoplasm_masks[0,win_cyto:-win_cyto,win_cyto:-win_cyto]
nuc = nuclear_masks[0,win_cyto:-win_cyto,win_cyto:-win_cyto]

dice_jaccard_indices(cyto, val, nuc)

[0.80490917, 0.73664939, 0.86436784, 0.84008527, 0.8562907, 0.90048212, 0.87222224, 0.86403787, 0.90625, 0.47875428, 0.82306039, 0.86963189, 0.81466031, 0.81662267, 0.83579725, 0.88251585, 0.84318608, 0.82974041, 0.84288144, 0.7116788, 0.82264149, 0.85488129, 0.88290155, 0.7756654, 0.92001659, 0.89930397, 0.90960991, 0.89569163] [0.89191098551748493, 0.8483570549991214, 0.92725030826140564, 0.91309385863267667, 0.92258252994449308, 0.94763544120311649, 0.93175074183976259, 0.92706041631409719, 0.95081967213114749, 0.64751026235945031, 0.90294365012615641, 0.93027071369975389, 0.89786535303776682, 0.89905591866376178, 0.91055507621512799, 0.93759193107795757, 0.91492234976367315, 0.90694878576580906, 0.91474299784549096, 0.83155650319829422, 0.90269151138716353, 0.92176386913229014, 0.93780957622454597, 0.87366167023554608, 0.95834232678609976, 0.94698265331053022, 0.95266567015445935, 0.94497607655502391]
Jaccard index is 0.834091 +/- 0.0838991
Dice index is 0.906904210871 +/- 0.057880

(0.83409059, 0.90690421087079309)